# Vorbereitung


In [1]:
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.17


# Verbinden zum Azure ML Workspace

In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: /home/nbuser/library/config.json
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FRZQ3XSKP to authenticate.
Interactive authentication successfully completed.
Workspace name: KI_Werkstatt
Azure region: westeurope
Subscription id: 659ab968-64ac-4ddc-bac9-f8a9549e7605
Resource group: RG_KI_Werkstatt


# GPU Cluster definieren 

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=1)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

Found existing compute target.
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-03-24T14:45:10.850000+00:00', 'creationTime': '2019-03-20T20:12:07.254297+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-03-20T20:13:44.388806+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


# Experiment vorbereiten

In [5]:
import os
from azureml.core import Experiment
from azureml.train.dnn import TensorFlow

exp = Experiment(workspace=ws, name='mnist')

script_params = {
    '--batch_size': 256,
    '--num_epochs': 200
}

tf_exp = TensorFlow(source_directory='./',
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train.py',
                    conda_packages=['scikit-learn'],
                    pip_packages=['keras','matplotlib'],
                    use_gpu=True)

# Experiment starten

In [6]:
run = exp.submit(tf_exp)

# Status abrufen

In [7]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…